# Track and Store Market Prices

Signal Tracking and Symbol Frequency Analysis

This notebook is part of a broader project that aims to evaluate the reliability and characteristics of cryptocurrency trading signals sent through a Telegram group. Specifically, this notebook focuses on identifying the most active and persistent trading symbols based on historical signal data.

## Purpose

The main objective of this notebook is to analyze the historical behavior of symbols present in the signal dataset in order to:

- Determine the first and last time each symbol was mentioned.
- Measure the total number of times each symbol was signaled.
- Calculate the number of days each symbol has been "active" in the dataset.
- Filter out symbols that do not meet basic consistency criteria.
- Rank the top symbols by their average signaling frequency per day.

These metrics allow us to prioritize symbols that have shown sustained usage over time and filter out low-frequency or short-lived mentions.

## Methodology

The process followed in this notebook includes the following steps:

1. **Data loading and cleaning**:
   - The dataset of cleaned trading signals (`telegram_signals_clean.csv`) is loaded.
   - Timestamps are parsed and validated.
   - Rows with invalid or missing timestamps are removed.

2. **Signal aggregation**:
   - For each symbol, the following metrics are computed:
     - `first_signal_date`
     - `last_signal_date`
     - `signal_count`
   - These values are saved in a separate file for reuse (`symbols_first_seen.csv`).

3. **Filtering criteria**:
   - Only symbols with at least 180 days between their first and last appearance are retained.
   - A minimum of 10 total signals is required for a symbol to be considered.

4. **Frequency scoring**:
   - For each filtered symbol, we compute `signals_per_day` as a measure of relative intensity.
   - The top 20 symbols are selected and saved as `top_20_symbols_by_frequency.csv`.

## Outputs

This notebook generates two key output files in the `data/clean/` directory:

- `symbols_first_seen.csv`: A summary table of all symbols with their activity metrics.
- `top_20_symbols_by_frequency.csv`: A list of the most actively used symbols, ranked by normalized frequency.

## Next Steps

The output of this notebook will be used in the next stage of the project (`04_signal_stats_analysis.ipynb`), where we will focus on deeper exploration of the selected symbols, including:

- Distribution of entry prices
- Target price patterns (`tp_40`, `tp_50`, etc.)
- Volatility and signal timing behavior
- Visual exploration of symbol characteristics

This separation of concerns ensures that the core statistical summary is reusable and modular for different downstream analyses.


In [ ]:
#install pybit
import sys
!{sys.executable} -m pip install pybit


In [ ]:
# Load libraries
import pandas as pd
from datetime import datetime
import sqlite3
from uuid import uuid4
from pybit.unified_trading import HTTP
from dotenv import load_dotenv
import os
import pandas as pd
import requests
import time
from datetime import datetime, timedelta

# Load environment variables
load_dotenv()
api_key = os.getenv("BYBIT_API_KEY")
api_secret = os.getenv("BYBIT_API_SECRET")

In [ ]:
# Load signals and compute statistics per symbol
import pandas as pd

# Load the cleaned signals dataset
signals_path = "../data/clean/telegram_signals_clean.csv"
df_signals = pd.read_csv(signals_path)

# Convert 'timestamp' to datetime format (with timezone handling)
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], errors="coerce", utc=True)

# Drop rows with invalid or missing timestamps (optional safeguard)
df_signals = df_signals.dropna(subset=["timestamp"])

# Group by symbol and compute first, last, and total signals
symbol_stats = df_signals.groupby("symbol").agg(
    first_signal_date=("timestamp", "min"),
    last_signal_date=("timestamp", "max"),
    signal_count=("timestamp", "count")
).reset_index()

# Sort by first appearance for readability
symbol_stats = symbol_stats.sort_values("first_signal_date")

# Display the result
print(symbol_stats.head(20))  # Or use display(symbol_stats)

# Save full stats to CSV
symbol_stats.to_csv("../data/clean/symbols_first_seen.csv", index=False)



In [ ]:
from datetime import datetime, timedelta

# Load all signals
signals_path = "../data/clean/telegram_signals_clean.csv"
df_signals = pd.read_csv(signals_path)
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], utc=True, errors="coerce")

# Filter last 6 months
six_months_ago = pd.Timestamp.utcnow() - pd.Timedelta(days=180)
recent_signals = df_signals[df_signals["timestamp"] >= six_months_ago]

# Get the top 30 most used symbols in the last 6 months
top_symbols = (
    recent_signals["symbol"]
    .value_counts()
    .head(30)  # antes: .head(10)
    .rename_axis("symbol")
    .reset_index(name="count")
)

# Save new top
top10_path = "../data/clean/top_30_final_scored_symbols.csv"
top_symbols.to_csv(top10_path, index=False)

print("Top 30 generated:", len(top_symbols))




In [ ]:
# Load history per symbol
symbol_stats_path = "../data/clean/symbols_first_seen.csv"
symbol_stats = pd.read_csv(symbol_stats_path)

# Convert date 
symbol_stats["last_signal_date"] = pd.to_datetime(symbol_stats["last_signal_date"], errors="coerce")
symbol_stats["symbol"] = symbol_stats["symbol"].str.upper()


In [ ]:
# Ensure that top_symbols has well-defined columns
top_symbols = (
    recent_signals["symbol"]
    .value_counts()
    .head(10)
    .rename_axis("symbol")  # convierte el índice en columna
    .reset_index(name="count")
)

# Create copy to enrich
top_10_combined = top_symbols.copy()
top_10_combined["symbol"] = top_10_combined["symbol"].str.upper()

# Ensure symbol_stats is also uppercase
symbol_stats["symbol"] = symbol_stats["symbol"].str.upper()

# Merge to get the last signal date per symbol 
top_10_combined = top_10_combined.merge(
    symbol_stats[["symbol", "last_signal_date"]],
    on="symbol",
    how="left"
)

#Reorder columns
top_10_combined = top_10_combined[["symbol", "count", "last_signal_date"]]

# Show results
print(top_10_combined)



In [ ]:
from pybit.unified_trading import HTTP
from datetime import datetime, timedelta

session = HTTP()
symbol = "MYROUSDT"

# Download parameters
interval = 60  # velas de 60 minutos
end = int(datetime.now().timestamp()) * 1000
start = int((datetime.now() - timedelta(days=30)).timestamp()) * 1000  # últimos 30 días

# Obtain historical data (OHLCV)
response = session.get_kline(
    category="spot",
    symbol=symbol,
    interval=str(interval),
    start=start,
    end=end
)

# Data access
data = response["result"]["list"]


In [ ]:
# TP Matching – TODAS las señales desde 2024 (sin top N)
import pandas as pd
from datetime import datetime

# Upload channel updates
updates_path = "../data/clean/telegram_updates_clean.csv"
updates_df = pd.read_csv(updates_path)
updates_df["timestamp"] = pd.to_datetime(updates_df["timestamp"], utc=True, errors="coerce")
updates_df["symbol"] = updates_df["symbol"].str.upper()

# Filter updates with a future date
today = pd.Timestamp.utcnow()
updates_df = updates_df[updates_df["timestamp"] <= today]

# Load full signals
signals_path = "../data/clean/telegram_signals_clean.csv"
df_signals = pd.read_csv(signals_path)
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], utc=True, errors="coerce")
df_signals["symbol"] = df_signals["symbol"].str.upper()

#Filter signals from 2024-01-01
start_date = pd.Timestamp(2024, 1, 1, tz="UTC")
df_signals = df_signals[df_signals["timestamp"] >= start_date].copy()

# Initialize columns for results
for tp in ["tp_40", "tp_60", "tp_80", "tp_100"]:
    df_signals[f"{tp}_result"] = None

#Tolerance to detect TP reached
TOLERANCE = 0.0005  # 0.05%

# Matching: compare each signal against subsequent updates
for i, row in df_signals.iterrows():
    symbol = row["symbol"]
    direction = row["direction"]
    signal_time = row["timestamp"]

    candidates = updates_df[
        (updates_df["symbol"] == symbol) &
        (updates_df["direction"] == direction) &
        (updates_df["timestamp"] > signal_time)
    ]

    for _, update in candidates.iterrows():
        hit_price = update["hit_price"]
        hit_time = update["timestamp"]

        for tp in ["tp_40", "tp_60", "tp_80", "tp_100"]:
            tp_val = row[tp]
            if pd.isna(df_signals.at[i, f"{tp}_result"]):
                if abs(hit_price - tp_val) / tp_val < TOLERANCE:
                    df_signals.at[i, f"{tp}_result"] = hit_time

# Save final file without filtering
output_path = "../telegram_signal_extractor/data/clean/signals_all_tp_results.csv"
df_signals.to_csv(output_path, index=False)

print(f"signals_all_tp_results.csv update {len(df_signals)} signals since 2024.")



In [ ]:
# Visual verification of the results
cols_to_show = [
    "symbol", "timestamp", "entry", "direction",
    "tp_40", "tp_40_result",
    "tp_60", "tp_60_result",
    "tp_80", "tp_80_result",
    "tp_100", "tp_100_result"
]

df_signals_top10[cols_to_show].head(30)


In [ ]:
# Route Configuration 
signals_path = "../data/clean/telegram_signals_clean.csv"
prices_dir = "../telegram_signal_extractor/data/bybit_prices"
output_path = "../data/clean/signals_tp_evaluation.csv"

# Load signals 
df_signals = pd.read_csv(signals_path)
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], utc=True, errors="coerce")
df_signals["symbol"] = df_signals["symbol"].str.upper()

# Optional: filter from 2024
df_signals = df_signals[df_signals["timestamp"] >= pd.Timestamp(2024, 1, 1, tz="UTC")].copy()

#Load real prices from Bybit (1h candles per symbol) 
symbols = df_signals["symbol"].unique()
df_prices = {}

for symbol in symbols:
    price_file = os.path.join(prices_dir, f"{symbol}.csv")
    if os.path.exists(price_file):
        df = pd.read_csv(price_file)
        df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
        df_prices[symbol] = df

# Function to check if a TP was reached
def check_tp_hit(price_df, signal_time, tp_value):
    df_future = price_df[price_df["timestamp"] > signal_time]
    return (df_future["high"] >= tp_value).any()

#Principal evaluation
results = []

for _, row in df_signals.iterrows():
    symbol = row["symbol"]
    signal_time = row["timestamp"]
    entry = row["entry"]
    direction = row["direction"]

    tps = {k: row[k] for k in ["tp_40", "tp_60", "tp_80", "tp_100"] if pd.notna(row[k])}

    if symbol not in df_prices:
        continue

    price_data = df_prices[symbol]
    result = {
        "symbol": symbol,
        "timestamp": signal_time,
        "entry": entry,
        "direction": direction
    }

    for tp_name, tp_val in tps.items():
        result[tp_name + "_hit"] = check_tp_hit(price_data, signal_time, tp_val)

    results.append(result)

# Save results
df_results = pd.DataFrame(results)
df_results.to_csv(output_path, index=False)

print(f"Save file: {output_path}")
print(df_results.head())

In [ ]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta, timezone
from pybit.unified_trading import HTTP

# Load cleaned signals
signals_path = "../data/clean/telegram_signals_clean.csv"
df = pd.read_csv(signals_path)
df["timestamp"] = pd.to_datetime(df["timestamp"], utc=True, errors="coerce")
df["symbol"] = df["symbol"].str.upper()

# Filter signals from 2024
df = df[df["timestamp"] >= pd.Timestamp(2024, 1, 1, tz="UTC")].copy()

# Get unique symbols
symbols = df["symbol"].unique()

# Setup Bybit session
session = HTTP()

# Output directory
output_dir = "../data/bybit_prices_1m"
os.makedirs(output_dir, exist_ok=True)

# Configuration
interval = "1"          # 1-minute candles
limit = 1000            # candles per API call
delay_between_calls = 1.5  # delay between requests in seconds

# Download data per symbol
for symbol in symbols:
    try:
        print(f"Fetching data for: {symbol}")

        # Start 2 minutes before the first signal for safety
        start_time = df[df["symbol"] == symbol]["timestamp"].min() - timedelta(minutes=2)
        end_time = datetime.now(timezone.utc)

        all_data = []

        while start_time < end_time:
            start_ts = int(start_time.timestamp()) * 1000
            end_ts = int((start_time + timedelta(minutes=limit)).timestamp()) * 1000

            response = session.get_kline(
                category="linear",
                symbol=symbol,
                interval=interval,
                start=start_ts,
                end=end_ts
            )

            data = response.get("result", {}).get("list", [])
            if not data:
                break

            df_chunk = pd.DataFrame(data, columns=[
                "timestamp", "open", "high", "low", "close", "volume", "turnover"
            ])
            df_chunk["timestamp"] = pd.to_datetime(df_chunk["timestamp"].astype(int), unit="ms", utc=True)
            df_chunk[["open", "high", "low", "close", "volume"]] = df_chunk[["open", "high", "low", "close", "volume"]].astype(float)

            all_data.append(df_chunk)

            # Prepare for next batch
            last_ts = df_chunk["timestamp"].max()
            last_ts = last_ts.tz_localize("UTC") if last_ts.tzinfo is None else last_ts
            start_time = last_ts + timedelta(minutes=1)

            time.sleep(delay_between_calls)

        if all_data:
            full_df = pd.concat(all_data, ignore_index=True)
            full_df.to_csv(os.path.join(output_dir, f"{symbol}.csv"), index=False)
            print(f"Saved {symbol}: {len(full_df)} rows")
        else:
            print(f"No data returned for {symbol}")

    except Exception as e:
        print(f"Error downloading {symbol}: {e}")


In [ ]:
import pandas as pd
import os
from datetime import datetime

# Load top 10 symbols
top10_path = "../data/clean/top_10_final_scored_symbols.csv"
top_10 = pd.read_csv(top10_path)
top_symbols = top_10["symbol"].str.upper().unique()

# Load clean signal data
signals_path = "../data/clean/telegram_signals_clean.csv"
df_signals = pd.read_csv(signals_path)
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], utc=True, errors="coerce")
df_signals["symbol"] = df_signals["symbol"].str.upper()

# Filter only top 10 symbols
df_signals = df_signals[df_signals["symbol"].isin(top_symbols)]

# Load Bybit historical price data
bybit_dir = "..data/bybit_prices"
price_data = {}

for symbol in top_symbols:
    price_file = os.path.join(bybit_dir, f"{symbol}.csv")
    if os.path.exists(price_file):
        df_price = pd.read_csv(price_file)
        df_price["timestamp"] = pd.to_datetime(df_price["timestamp"])
        price_data[symbol] = df_price

# Function to detect TP hit time 
def get_tp_hit_time(df_price, signal_time, tp_value, direction):
    df_price["timestamp"] = pd.to_datetime(df_price["timestamp"], utc=True, errors="coerce")
    future_prices = df_price[df_price["timestamp"] >= signal_time]
    if future_prices.empty:
        return "No price data"

    if direction.lower() == "long":
        hit = future_prices[future_prices["high"] >= tp_value]
    elif direction.lower() == "short":
        hit = future_prices[future_prices["low"] <= tp_value]
    else:
        return "Invalid direction"

    if not hit.empty:
        return hit.iloc[0]["timestamp"].strftime("%Y-%m-%d %H:%M:%S")
    else:
        return "tp_40 not reached"

#Evaluate TP40 for each signal
results = []
for _, row in df_signals.iterrows():
    symbol = row["symbol"]
    if symbol not in price_data:
        continue

    signal_time = row["timestamp"]
    entry = row["entry"]
    tp_40 = row["tp_40"]
    direction = row["direction"]

    result = {
        "symbol": symbol,
        "timestamp": signal_time,
        "entry": entry,
        "tp_40": tp_40,
        "direction": direction
    }

    result["tp_40_result"] = get_tp_hit_time(price_data[symbol], signal_time, tp_40, direction)
    results.append(result)

# --- 6. Save results to CSV ---
df_result = pd.DataFrame(results)
output_path = "../data/clean/signals_tp40_time_v2.csv"
df_result.to_csv(output_path, index=False)
print(f" Saved TP40 evaluation to: {output_path}")


In [ ]:
#Load top 10 symbols 
top10_path = "../data/clean/top_10_final_scored_symbols.csv"
top_10 = pd.read_csv(top10_path)
top_symbols = top_10["symbol"].str.upper().unique()

# oad clean signal data
signals_path = "../data/clean/telegram_signals_clean.csv"
df_signals = pd.read_csv(signals_path)
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], utc=True, errors="coerce")
df_signals["symbol"] = df_signals["symbol"].str.upper()

# Filter only top 10 symbols
df_signals = df_signals[df_signals["symbol"].isin(top_symbols)]

# Load Bybit historical price data
bybit_dir = "..data/bybit_prices"
price_data = {}

for symbol in top_symbols:
    price_file = os.path.join(bybit_dir, f"{symbol}.csv")
    if os.path.exists(price_file):
        df_price = pd.read_csv(price_file)
        df_price["timestamp"] = pd.to_datetime(df_price["timestamp"], utc=True, errors="coerce")
        price_data[symbol] = df_price

# Function to detect TP hit time ---
def get_tp_hit_time(df_price, signal_time, tp_value, direction):
    future_prices = df_price[df_price["timestamp"] >= signal_time]
    if future_prices.empty:
        return "No price data"

    if direction.lower() == "long":
        hit = future_prices[future_prices["high"] >= tp_value]
    elif direction.lower() == "short":
        hit = future_prices[future_prices["low"] <= tp_value]
    else:
        return "Invalid direction"

    if not hit.empty:
        return hit.iloc[0]["timestamp"].strftime("%Y-%m-%d %H:%M:%S")
    else:
        return "TP not reached"

# Evaluate all TP levels for each signal ---
results = []
for _, row in df_signals.iterrows():
    symbol = row["symbol"]
    if symbol not in price_data:
        continue

    signal_time = row["timestamp"]
    entry = row["entry"]
    direction = row["direction"]

    result = {
        "symbol": symbol,
        "timestamp": signal_time,
        "entry": entry,
        "direction": direction,
        "tp_40": row["tp_40"],
        "tp_60": row["tp_60"],
        "tp_80": row["tp_80"],
        "tp_100": row["tp_100"],
    }

    price_df = price_data[symbol]

    result["tp_40_result"] = get_tp_hit_time(price_df, signal_time, row["tp_40"], direction)
    result["tp_60_result"] = get_tp_hit_time(price_df, signal_time, row["tp_60"], direction)
    result["tp_80_result"] = get_tp_hit_time(price_df, signal_time, row["tp_80"], direction)
    result["tp_100_result"] = get_tp_hit_time(price_df, signal_time, row["tp_100"], direction)

    results.append(result)

# Save results to CSV 
df_result = pd.DataFrame(results)
output_path = "../data/clean/signals_all_tp_results.csv"
df_result.to_csv(output_path, index=False)
print(f"Saved full TP evaluation to: {output_path}")


In [ ]:
import pandas as pd

# Local path to the updates file
updates_path = "../data/clean/telegram_updates_clean.csv"
updates_df = pd.read_csv(updates_path)

#Convert dates
df_signals["timestamp"] = pd.to_datetime(df_signals["timestamp"], utc=True, errors="coerce")
updates_df["timestamp"] = pd.to_datetime(updates_df["timestamp"], utc=True, errors="coerce")

#Initialize empty columns
for tp in ["tp_40", "tp_60", "tp_80", "tp_100"]:
    df_signals[f"{tp}_result"] = None

#Define tolerance
TOLERANCE = 0.001

# Matching
for i, row in df_signals.iterrows():
    symbol = row["symbol"]
    direction = row["direction"]
    signal_time = row["timestamp"]

    candidates = updates_df[
        (updates_df["symbol"] == symbol) &
        (updates_df["direction"] == direction) &
        (updates_df["timestamp"] > signal_time)
    ]

    for _, update in candidates.iterrows():
        hit_price = update["hit_price"]
        ts = update["timestamp"]

        for tp in ["tp_40", "tp_60", "tp_80", "tp_100"]:
            target = row[tp]
            if df_signals.at[i, f"{tp}_result"] is None:
                if abs(hit_price - target) / target < TOLERANCE:
                    df_signals.at[i, f"{tp}_result"] = ts

# Save updated file
output_path = "../data/clean/signals_all_tp_results.csv"
df_signals.to_csv(output_path, index=False)

print("File signals_all_tp_results.csv updated with actual results .")

